## Langevin dynamics: introduction with simulations

Topics:
- Perform simple simulations of Langevin dynamics
- Verify the expected equilibrium properties
- Probe the dynamics with correlation functions

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import tidynamics

### A super mega crash course on Python

This introduction uses Python.

Let's go!

In [ ]:
# Variables

a = 1
print(a, type(a))
a = '1'
print(a, type(a))
a = 1.0
print(a, type(a))


In [ ]:
# Loops

for a in [1, '1', 1.0]:
    print(a, type(a))

In [ ]:
# Functions

def my_function(x, y):
    return x+y

print(my_function(1, 1))
print(my_function('one ', 'two'))
print(my_function(1.0, 2.0))
print(my_function(1.0, 'two'))

## Arrays in Python

In Python, the most convenient data type for numerical data is the NumPy array.

In [ ]:
my_array = np.ones(5)
print(my_array)
my_array[0] = 10
print(my_array)
print(2*my_array)
print(np.sin(my_array))

## The Langevin equation

The Langevin equation for the velocity (Ornstein-Uhlenbeck process)

$$\dot v = - \gamma v + \sqrt{2 \gamma T} \xi$$

where $\gamma$ is the friction coefficient, $T$ is the temperature ($k_B=1$) and
$\xi$ is gaussian white noise.

Questions:
1. How to represent the noise numerically?
2. How to do a simulation of this equation?

### White noise

Noise is generated from a "pseudo random number generator" (RNG or PRNG).

In [ ]:
sample_data = np.random.normal(size=100)

In [ ]:
plt.figure()
count, bins, patches = plt.hist(sample_data, bins=32, normed=True)
plt.plot(bins, np.exp(-bins**2/2)/np.sqrt(2*np.pi))

## What about the correlations?

A definining feature of the noise $\xi$ is its autocorrelation:

$$\langle \xi(t_1) \xi(t_2) \rangle = 2 T \gamma \delta(t_1-t_2)$$

**First exercise:** Compute the autocorrelation of the noise.

In discrete time, use

$$\langle \xi(t_1) \xi(t_2) \rangle = \frac{1}{\# \mathrm{ samples}} \sum_{i, j} \xi(i) \xi(j)$$

where the sum is taken over the i and j that obey $i-j = t_1-t_2$

In [ ]:
N_data_points = 1000
noise = np.random.normal(size=N_data_points)

autocorrelation = np.zeros(2*N_data_points - 1)
autocorrelation_count = np.zeros(2*N_data_points - 1)
center_point = N_data_points

for i in range(N_data_points):
    for j in range(N_data_points):
        index = center_point + i - j - 1
        autocorrelation[index] += noise[i]*noise[j]
        autocorrelation_count[index] += 1
        

autocorrelation /= autocorrelation_count

In [ ]:
plt.figure()
plt.plot(autocorrelation)

In [ ]:
x_axis = center_point + np.arange(len(noise)) - 1
plt.plot(x_axis, tidynamics.acf(noise))

## Solutions to the Langevin equation

$$v(t+dt) = v(t) + \int_t^{t+dt} dt' ~ \left[ -\gamma v(t') + \sqrt{2\gamma T} \xi(t')\right]$$

The Euler-Maruyama algorithm is the simplest stochastic integrator:

$$v(t+dt) \approx v(t) - \gamma v(t) dt + \sqrt{2\gamma T dt} \Gamma$$

In [ ]:
def euler_step(v, gamma, T, dt):
    return v - gamma*v*dt + np.sqrt(2*gamma*T*dt)*np.random.normal()


In [ ]:
data = [] # Create an empty list

γ = 0.1
dt = 0.01
T = 2

v = 0 # Initial condition

for i in range(1000):
    v = euler_step(v, γ, T, dt)

for i in range(10000):
    for j in range(10):
        v = euler_step(v, γ, T, dt)
    data.append(v) # Sample every 10 dt
data = np.array(data)

In [ ]:
plt.figure()

plt.plot(data)
plt.xlabel('Time') ; plt.ylabel('velocity')

In [ ]:
plt.figure()
count, bins, patches = plt.hist(data, bins=32, normed=True)
plt.plot(bins, np.exp(-bins**2/(2*T))/np.sqrt(2*np.pi*T))
plt.xlabel('velocity') ; plt.title('Histogram of velocity')